In [ ]:
# Login to ArcGIS Online

def login_AGOL():
    username = input("Enter your ArcGIS Online username:")
    password = getpass.getpass("Enter your ArcGIS Onine password:")
    gis = GIS("https://www.arcgis.com",username,password)
    return gis

gis = login_AGOL()